# Cookbook Template

This gets you set up with a basic PluginInstanceRedisInterface that can communicate with your Nanome workspace from a Jupyter Notebook

Note that anything involving Redis is specific to the Cookbook project, and is not used for traditional plugins.

For more info on building regular plugins, see the 'Plugins' notebook.


## Setup.
1) Open up a Room in Nanome.
2) Navigate to the Stacks menu. If your Cookbook has been sucessfully deployed, you should have a "Cookbook" entry in your list
2) Start the Plugin, and the run button should say "Live". This mean the room is ready to accept requests from your notebook

In [ ]:
import os
from nanome.beta.redis_interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = os.environ.get("REDIS_HOST")
redis_port = os.environ.get("REDIS_PORT")
redis_password = os.environ.get("REDIS_PASSWORD")
redis_channel = "12345"  # Use room id code you receive when activating cookbook plugin

plugin = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
plugin.connect()
print('Plugin Connected')

In [ ]:
# Load a TYL into your workspace
tyl_pdb = 'test_data/1tyl.pdb'
plugin.send_files_to_load([tyl_pdb])


In [ ]:
# Get new complex from workspace
ws = plugin.request_workspace()
comp = ws.complexes[-1]

In [ ]:
# Lets make the whole complex blue
from nanome.util import Color
color = Color.Blue()

for residue in comp.residues:
    residue.ribbon_color = color
    for atom in residue.atoms:
        atom.atom_color = color
plugin.update_structures_deep([comp])

In [ ]:
# Lets reapply the default Element color scheme.
from nanome.util import enums
color_scheme = enums.ColorScheme.Element
color_scheme_target = enums.ColorSchemeTarget.All

# Color schemes apply to all selected atoms
for atom in comp.atoms:
    atom.selected = True
plugin.update_structures_deep([comp])
plugin.apply_color_scheme(color_scheme, color_scheme_target)

# Pull latest changes into memory
[comp] = plugin.request_complexes([comp.index])

In [ ]:
# Lets do some fun stuff

# Create useful functions we can reuse later
from nanome.api import structure
from scipy.spatial import KDTree
import itertools

def select_atoms(atom_list):
    for atm in atom_list:
        atm.selected = True

def render_surface_around_atoms(comp, atom_list):
    for atm in comp.atoms:
        atm.surface_rendering = atm in atom_list
    comp._surface_dirty = True


def get_neighboring_atoms(target_reference: structure.Complex, selected_atoms: list, radius=5):
    """Use KDTree to find target atoms within site_size radius of selected atoms.
    
    returns a list of atoms within `site_size` angstroms of the ligand.
    """
    mol = next(
        mol for i, mol in enumerate(target_reference.molecules)
        if i == target_reference.current_frame)
    ligand_positions = [atom.position.unpack() for atom in selected_atoms]
    target_atoms = itertools.chain(*[ch.atoms for ch in mol.chains if not ch.name.startswith("H")])
    target_tree = KDTree([atom.position.unpack() for atom in target_atoms])
    target_point_indices = target_tree.query_ball_point(ligand_positions, radius)
    near_point_set = set()
    for point_indices in target_point_indices:
        for point_index in point_indices:
            near_point_set.add(tuple(target_tree.data[point_index]))
    neighbor_atoms = []
    for targ_atom in mol.atoms:
        if targ_atom.position.unpack() in near_point_set:
            neighbor_atoms.append(targ_atom)
    return neighbor_atoms

In [ ]:
"""Render the surface around the pocket surrounding the TYL ligand."""

from nanome.util import enums

ligand_res = next(res for res in comp.residues if res.name.upper() == 'TYL')
ligand_atoms = list(ligand_res.atoms)

neighbor_radius = 5
ligand_pocket = get_neighboring_atoms(comp, ligand_atoms, neighbor_radius)

comp.set_all_selected(False)
select_atoms(ligand_pocket)
render_surface_around_atoms(comp, ligand_pocket)

plugin.update_structures_deep([comp])
plugin.apply_color_scheme(enums.ColorScheme.DonorAcceptor, enums.ColorSchemeTarget.All)